In [1]:
import pennylane as qml
import pandas as pd
import qutip
from pennylane import numpy as np
from random import uniform
from math import pi

n_wires = 4
n_layers = 2
iteracoes = 5
n_targets = 10

def SimplifiedTwoDesign(n_layers, n_wires, n_targets):
    
    #Pesos
    init_weights = [uniform(0,2*pi), uniform(0,2*pi), uniform(0,2*pi), uniform(0,2*pi)]

    weights_layer1 = [[uniform(0,2*pi), uniform(0,2*pi)],
                      [uniform(0,2*pi), uniform(0,2*pi)],
                      [uniform(0,2*pi), uniform(0,2*pi)]]
    weights_layer2 = [[uniform(0,2*pi), uniform(0,2*pi)],
                      [uniform(0,2*pi), uniform(0,2*pi)],
                      [uniform(0,2*pi), uniform(0,2*pi)]]
    weights_layer3 = [[uniform(0,2*pi), uniform(0,2*pi)],
                      [uniform(0,2*pi), uniform(0,2*pi)],
                      [uniform(0,2*pi), uniform(0,2*pi)]]   

        
    #Cria dados
    def normaliza(v):
        v = v / (np.linalg.norm(v))    
        return v

    def cria_alvos(n_targets, n_wires):
        target_states = []
        for i in range(n_targets):
            target_states.append(normaliza(np.random.rand(1, 2**(n_wires))))
        return target_states



    @qml.qnode(dev)
    def circuit(init_weights, weights):
        qml.SimplifiedTwoDesign(initial_layer_weights=init_weights, weights=weights, wires=range(n_wires))
        return qml.state()
    

    def ket_in_amplt(v):
        aux = []
        for i in range(len(v)):
            v[i] = np.dot(v[i],v[i])
            v[i] = np.linalg.norm(v[i])
            aux.append(float(v[i].real))
        aux = np.array(aux)    
        return aux
      


    print("\n")
    weights = [weights_layer1, weights_layer2, weights_layer3]
    print("\nVetor de matrizes de peso:", weights)
    print(qml.draw(circuit, expansion_strategy="device")(init_weights, weights))
    ket = circuit(init_weights, weights)
    amplt = ket_in_amplt(ket)
    print("\nEstado resultado do circuito: ", ket)
    target_states = cria_alvos(n_targets, n_wires)



    def fidelity(target_states, amplt):
        fidelity_sum = 0
        for target_state in target_states:
            output_probs = amplt
            fidelity = np.vdot(target_state, output_probs) ** 2
            fidelity_sum += fidelity
    
        average_fidelity = fidelity_sum / len(target_states)
        return average_fidelity
    

    def compute_Q_ptrace(ket, N):
        ket = qutip.Qobj(ket, dims=[[2]*(N), [1]*(N)]).unit()
        entanglement_sum = 0
        for k in range(N):
          rho_k_sq = ket.ptrace([k])**2
          entanglement_sum += rho_k_sq.tr()  
    
        Q = 2*(1 - (1/N)*entanglement_sum)
        return Q




    Q = compute_Q_ptrace(ket, n_wires)
    #print("\nGrau de Emaranhamento: {}".format(Q))

    E = fidelity(target_states, amplt)
    #print("Expressabilidade: {}".format(E))

    #print("\n\nEstado gerado: {}".format(ket))
    #print("\n\n\n\n\n")

    k=[Q,E]
    return k


dev = qml.device('default.qubit', wires=n_wires)


k=[]
for i in range(iteracoes):
    print("\n=============")
    print("= Modelo: {} =".format(i+1))
    print("=============\n")
    k.append(SimplifiedTwoDesign(n_layers, n_wires, n_targets))


print("\n\nResultados:\n")
for i in range(iteracoes):
    print(" Modelo {} \n Grau de emaranhamento: {} \n Expressabilidade: {} \n".format(i+1,k[i][0],k[i][1]))


def pand(k):
    eman=[]
    exp=[]

    for i in range(iteracoes):
        eman.append(k[i][0])
    for i in range(iteracoes):
        exp.append(k[i][1]) 

    dic = {"Grau de Emaranhamento": eman, "Expressabilidade":exp}
    df = pd.DataFrame(dic)
    return df

df = pand(k)
df.describe()



= Modelo: 1 =




Vetor de matrizes de peso: [[[2.8724735893295446, 5.583534180968832], [5.429064590557219, 6.005257109296857], [2.8723861623314106, 1.1959652790670658]], [[2.4856549000868324, 1.4005822441955997], [0.5999638653939915, 2.924185588611008], [2.867648604087188, 0.7831335798603521]], [[4.223616416625625, 1.9161870688637346], [0.07094110300707689, 2.8309634236128014], [1.930508491852998, 5.1148720492916135]]]


TypeError: list indices must be integers or slices, not tuple